In [4]:
import pandas as pd
import numpy as np

from urllib.parse import quote    
from urllib.request import urlopen

import sys
import requests
from bs4 import BeautifulSoup 

# Read Data

In [5]:
bus = pd.read_csv('BUS_STATION_BOARDING_MONTH_202001.csv',encoding = 'euc-kr')
bus.rename(columns={'역명':'정류장명',
                   '버스정류장ARS번호':'ARSID'},inplace=True)
#bus['ARSID'] = bus['ARSID'].astype('int64') # ARSID에 문자열 정보가 존재

In [6]:
subway = pd.read_csv('CARD_SUBWAY_MONTH_201912.csv',encoding = 'euc-kr')

In [7]:
bus_coord = pd.read_csv('서울시버스정류소좌표.csv')

In [16]:
kor_address = pd.read_csv('korea_address.csv')
kor_address.columns = kor_address.iloc[0,:].tolist()
kor_address.drop(index=0,inplace=True)

In [47]:
area_code = pd.read_csv('지역코드.csv')

area_code.columns = ['code','지역구']
pre_code = pd.DataFrame({'code':[1],
              '지역구':['종로구']})

area_code = area_code.append(pre_code)

In [138]:
bus_coord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11180 entries, 0 to 11179
Data columns (total 5 columns):
ARSID    11180 non-null int64
표준ID     11180 non-null int64
정류장명     11180 non-null object
X좌표      11180 non-null float64
Y좌표      11180 non-null float64
dtypes: float64(2), int64(2), object(1)
memory usage: 436.8+ KB


In [8]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18333 entries, 0 to 18332
Data columns (total 7 columns):
사용일자      18333 non-null int64
노선명       18333 non-null object
역ID       18333 non-null int64
역명        18333 non-null object
승차총승객수    18333 non-null int64
하차총승객수    18333 non-null int64
등록일자      18333 non-null int64
dtypes: int64(5), object(2)
memory usage: 1002.7+ KB


In [139]:
#bus arsid 는 문자열이 껴있는 경우가 있음
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192916 entries, 0 to 1192915
Data columns (total 11 columns):
사용일자         1192916 non-null int64
노선ID         1192916 non-null int64
노선번호         1192916 non-null object
노선명          1192916 non-null object
표준버스정류장ID    1192916 non-null int64
ARSID        1192916 non-null object
역ID          1192916 non-null int64
정류장명         1192916 non-null object
승차총승객수       1192916 non-null int64
하차총승객수       1192916 non-null int64
등록일자         1192916 non-null int64
dtypes: int64(7), object(4)
memory usage: 100.1+ MB


In [19]:
kor_address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21662 entries, 1 to 21662
Data columns (total 11 columns):
시도            21662 non-null object
시군구           21662 non-null object
행정구역명         21662 non-null object
행정동(행정기관명)    21662 non-null object
법정동           21662 non-null object
행정구역코드        21584 non-null object
행정기관코드        21662 non-null object
행정기관 생성일      21662 non-null object
법정동코드         21662 non-null object
관할지역          1664 non-null object
행정동 영문명칭      21661 non-null object
dtypes: object(11)
memory usage: 2.0+ MB


In [25]:
area_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
1      24 non-null int64
종로구    24 non-null object
dtypes: int64(1), object(1)
memory usage: 512.0+ bytes


# subway station name preprocessing

- api로 역사정보를 parsing하기 위해서는 정제된 이름을 경로에 지정해서 넣어줘야 함
- 예를들면 : /서울역 (x) , /서울 (o) -> 오로지 서울역만 뒤에 역명이 붙어서 존재
- 또한 : /잠실(송파구청) (x) , /잠실(0)
- 그리고 현재 api데이터에 우이신설선은 업데이트 되지 않은듯

In [62]:
subway_name0 = subway['역명'].unique().tolist()

station_name = []
for i in subway_name0:
    if '(' in i :
        name_ls0 = i.split('(')
        name = name_ls0[0]
        station_name.append(name)
    else:
        station_name.append(i)

station_name2 = []
for i in station_name:
    if i[-1] == '역':
        station_name2.append(i[:-1])
    else:
        station_name2.append(i)

In [63]:
api_key = '436261446967796537367159686464'
base_url = 'http://swopenAPI.seoul.go.kr/api/subway/{}/xml/stationInfo/0/100/{}'
station_info = pd.DataFrame()
error_num = []
for j in range(len(station_name2)):
    try:
        req = requests.get(base_url.format(api_key,station_name2[j]), headers = {'user-agent': \
                                                ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36')})

        soup = BeautifulSoup(req.content,'html.parser')
        if '해당하는 데이터가 없습니다' in soup.text:
            print(station_name2[j] + ' 수집정보 x')
            error_num.append(j)
            pass
        else:
            
            main_station = []
            statn_id = []
            subway_id = []
            subway_nm = []
            statnf_id = []
            statnf_name = []
            statnt_id = []
            statnt_name = []
            oper = []
            zipno = [] #??
            address_bass = []
            address_detail = []
            statn_tel = []
            x_code = []
            y_code = []
            transit = []

            for_loop = len(soup.find_all('statnnm'))

            for i in range(for_loop):
                main_station.append(soup.find_all('statnnm')[i].text)
                statn_id.append(soup.find_all('statnid')[i].text)
                subway_id.append(soup.find_all('subwayid')[i].text)
                subway_nm.append(soup.find_all('subwaynm')[i].text)
                statnf_id.append(soup.find_all('statnfid')[i].text)
                statnf_name.append(soup.find_all('statnfnm')[i].text)
                statnt_id.append(soup.find_all('statntid')[i].text)
                statnt_name.append(soup.find_all('statntnm')[i].text)
                oper.append(soup.find_all('operpblinstt')[i].text)
                zipno.append(soup.find_all('zipno')[i].text)
                address_bass.append(soup.find_all('adresbass')[i].text)
                address_detail.append(soup.find_all('adresdetail')[i].text)
                statn_tel.append(soup.find_all('telno')[i].text)
                x_code.append(soup.find_all('subwayxcnts')[i].text)
                y_code.append(soup.find_all('subwayycnts')[i].text)
                transit.append(soup.find_all('trnsitco')[i].text)

            station_info0 = pd.DataFrame({'statn_id' : statn_id,
                                          'main_station' : main_station,
                                          'subway_id' : subway_id,
                                          'subway_nm' : subway_nm,
                                          'statnf_id' : statnf_id,
                                          'statnf_name' : statnf_name,
                                          'statnt_id' : statnt_id,
                                          'statnt_name' : statnt_name,
                                          'oper' : oper,
                                          'zipno' : zipno,
                                          'address_bass' : address_bass,
                                          'address_detail' : address_detail,
                                          'x_code' : x_code,
                                          'y_code' : y_code,
                                          'transit' : transit})

            station_info = station_info.append(station_info0)
            print('--------------'+ station_name2[j] + ' 역 완료----------------')
    except:
        print(station_name2[j] + ' 역 request error')
        pass

--------------종각 역 완료----------------
--------------종로3가 역 완료----------------
--------------종로5가 역 완료----------------
--------------동대문 역 완료----------------
--------------신설동 역 완료----------------
--------------제기동 역 완료----------------
--------------청량리 역 완료----------------
--------------동묘앞 역 완료----------------
--------------시청 역 완료----------------
--------------을지로입구 역 완료----------------
--------------을지로3가 역 완료----------------
--------------을지로4가 역 완료----------------
--------------동대문역사문화공원 역 완료----------------
--------------신당 역 완료----------------
--------------상왕십리 역 완료----------------
--------------왕십리 역 완료----------------
--------------한양대 역 완료----------------
--------------뚝섬 역 완료----------------
--------------성수 역 완료----------------
--------------건대입구 역 완료----------------
--------------구의 역 완료----------------
--------------강변 역 완료----------------
--------------잠실나루 역 완료----------------
--------------잠실 역 완료----------------
--------------잠실새내 역 완료----------------
--------------종

--------------개포동 역 완료----------------
--------------대모산입구 역 완료----------------
--------------서울숲 역 완료----------------
--------------압구정로데오 역 완료----------------
--------------강남구청 역 완료----------------
--------------선정릉 역 완료----------------
--------------가천대 역 완료----------------
--------------태평 역 완료----------------
--------------모란 역 완료----------------
--------------야탑 역 완료----------------
--------------서현 역 완료----------------
--------------수내 역 완료----------------
--------------정자 역 완료----------------
--------------미금 역 완료----------------
--------------오리 역 완료----------------
--------------이매 역 완료----------------
--------------보정 역 완료----------------
--------------죽전 역 완료----------------
--------------구성 역 완료----------------
--------------신갈 역 완료----------------
--------------기흥 역 완료----------------
--------------상갈 역 완료----------------
--------------청명 역 완료----------------
--------------영통 역 완료----------------
--------------망포 역 완료----------------
--------------매탄권선 역 완료--------------

--------------보라매 역 완료----------------
--------------신풍 역 완료----------------
--------------남구로 역 완료----------------
--------------철산 역 완료----------------
--------------광명사거리 역 완료----------------
--------------천왕 역 완료----------------
--------------까치울 역 완료----------------
--------------부천종합운동장 역 완료----------------
--------------춘의 역 완료----------------
--------------신중동 역 완료----------------
--------------부천시청 역 완료----------------
--------------상동 역 완료----------------
--------------삼산체육관 역 완료----------------
--------------굴포천 역 완료----------------
--------------부평구청 역 완료----------------
--------------암사 역 완료----------------
--------------강동구청 역 완료----------------
--------------몽촌토성 역 완료----------------
--------------석촌 역 완료----------------
--------------송파 역 완료----------------
--------------문정 역 완료----------------
--------------장지 역 완료----------------
--------------복정 역 완료----------------
--------------산성 역 완료----------------
--------------남한산성입구 역 완료----------------
--------------단대오거리 역 

In [71]:
station_info.to_csv('station_info.csv')
station_info.head()

statn_id main_station subway_id subway_nm   statnf_id statnf_name  \
0  1001000131           종각      1001       1호선  1001000130        종로3가   
0  1003000329         종로3가      1003       3호선  1003000328          안국   
1  1005000534         종로3가      1005       5호선  1005000533         광화문   
2  1001000130         종로3가      1001       1호선  1001000129        종로5가   
0  1001000129         종로5가      1001       1호선  1001000128         동대문   

    statnt_id statnt_name   oper   zipno    address_bass address_detail  \
0  1001000132          시청  서울메트로  110121  서울특별시 종로구 종로1가             44   
0  1003000330       을지로3가  서울메트로  110330   서울특별시 종로구 돈의동             46   
1  1005000535       을지로4가   도시철도  110330   서울특별시 종로구 돈의동             46   
2  1001000131          종각  서울메트로  110330   서울특별시 종로구 돈의동             46   
0  1001000130        종로3가  서울메트로  110125  서울특별시 종로구 종로5가          124-8   

        x_code       y_code transit  
0  198520.9972  452310.4761       1  
0     199270.9     452469.7       3  
1  199127.4283  452568.6269       3  
2  199296.4693   452334.288       3  
0  200171.3944  452385.8684       1

In [65]:
#데이터상으로 아직 우이신설선 데이터 x
for i in error_num:
    print(station_name2[i])

총신대입구
쌍용
삼동
경기광주
초월
곤지암
신둔도예촌
이천
부발
세종대왕릉
여주
이수
북한산우이
솔밭공원
4.19민주묘지
가오리
화계
삼양
삼양사거리
솔샘
북한산보국문
정릉


In [66]:
# station 구 추출
'''
각 시별로 구의 위치가 다르고 또한 구가 없는 지역도 있으므로 해당 지역들은 따로 처리를 하고
서울 지역의 구만 추출
'''
for i in range(len(station_info)):
    if len(station_info['address_bass'].iloc[i].split(' ')) != 3:
        print(station_info['address_bass'].iloc[i])
    else:
        pass

강북구 우이동
서울특별시 동대문구 왕산로 214
서울특별시 동대문구 왕산로 214
서울특별시 동대문구 왕산로 214
서울특별시 성동구 왕십리광장로 17
서울특별시 성동구 완십리광장로 17
서울특별시 강남구 강남대로 지하396
서울특별시 마포구 양화로 지하188
경기도 고양시 덕양구 지축동
서울특별시 성동구 동호로 지하21
서울특별시 서초구 남부순환로 지하2585
성북구 동소문동
서울특별시 성북구 동선동 4가
서울특별시 용산구 한강대로 405
서울특별시 용산구 이촌로71길 42
서울특별시 용산구 한강대로23길 55
경기도 안양시 만안구 석수동
경기도 안양시 만안구 안양1동
경기도 안양시 만안구 안양6동
경기도 수원시 장안구 율전동
경기도 수원시 장안구 화서2동
경기도 수원시 팔달구 매산로1가 18
경기도 수원시 권선구 매산로1가
경기도 수원시 권선구 장지동
경기도 평택시 진위면 하북리
충청남도 천안시 성환읍 성환리
충청남도 천안시 직산읍 모시리
경기도 부천시 원미구 역곡동
경기도 부천시 소사구 심곡본동
경기도 부천시 소사구 송내동
인천광역시 부평구 광장로 지하15
인천광역시 남구 주안동 125
인천광역시 중구 북성동 2가 14
경기도 부천시 소사구 소사본2동
경기도 부천시 소사구 송내동
서울특별시 용산구 서빙고로 238
서울특별시 용산구 독서당로6길 12-13
서울특별시 성동구 고산자로 123
서울특별시 노원구 석계로 98-2
경기도 의정부시 평화로 363
경기도 안산시 상록구 대야미동
경기도 안산시 상록구 건건동
경기도 안산시 상록구 일동
경기도 안산시 상록구 이동
경기도 안산시 단원구 고잔동
경기도 안산시 단원구 고잔동
경기도 안산시 단원구 초지동
경기도 안산시 단원구 초지동
경기도 안산시 단원구 원곡동
경기도 안산시 단원구 신길동
경기도 시흥시 역전로 430
경기도 안양시 동안구 관양2동
경기도 안양시 동안구 부림동
경기도 안앙시 동안구 호계동
서울특별시 성동구 왕십리로 지하77
서울특별시 강남구 압구정로 지하402
서울특별시 강남구 학동로 지하346
서울

In [69]:
ku_ls = []
for i in range(len(station_info)):
    if '서울' not in station_info['address_bass'].iloc[i]:
        ku_ls.append(np.nan)
    else:
        ku_ls.append(station_info['address_bass'].iloc[i].split(' ')[1])
station_info['ku'] = ku_ls

# bus preprocessing

In [76]:
bus.head()

사용일자      노선ID  노선번호                   노선명  표준버스정류장ID  ARSID      역ID  \
0  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034  04133    72645   
1  20200101  41110061  6616         6616(철산동~온수동)  116000120  17210    10202   
2  20200101  41110061  6616         6616(철산동~온수동)  116000080  17166  8001850   
3  20200101  41110061  6616         6616(철산동~온수동)  116000083  17169  9004386   
4  20200101  41110061  6616         6616(철산동~온수동)  116000085  17171  9004384   

              정류장명  승차총승객수  하차총승객수      등록일자  
0             왕십리역       0      13  20200104  
1     현대아파트.개봉중앙시장     153      92  20200104  
2            경인중학교     171      46  20200104  
3  오류1동주민센터.오류문화센터      87      16  20200104  
4             오류동역     129      46  20200104

In [77]:
pd.merge(bus,bus_coord, on='정류장명',how='left').head()

사용일자      노선ID  노선번호                   노선명  표준버스정류장ID ARSID_x    역ID  \
0  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034   04133  72645   
1  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034   04133  72645   
2  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034   04133  72645   
3  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034   04133  72645   
4  20200101  41110061  6616         6616(철산동~온수동)  116000120   17210  10202   

           정류장명  승차총승객수  하차총승객수      등록일자  ARSID_y         표준ID         X좌표  \
0          왕십리역       0      13  20200104   4133.0  103000034.0  127.036269   
1          왕십리역       0      13  20200104   4134.0  103000035.0  127.036193   
2          왕십리역       0      13  20200104   4135.0  103000036.0  127.034901   
3          왕십리역       0      13  20200104   4781.0  105000472.0  127.037685   
4  현대아파트.개봉중앙시장     153      92  20200104  17209.0  116000119.0  126.856039   

         Y좌표  
0  37.559988  
1  37.560298  
2  37.560770  
3  37.560936  
4  37.490892

In [75]:
bus_coord.head()

ARSID       표준ID           정류장명         X좌표        Y좌표
0   1001  100000001        종로2가사거리  126.987786  37.569764
1   1002  100000002    창경궁.서울대학교병원  126.996520  37.579179
2   1003  100000003      명륜3가.성대입구  126.998290  37.582709
3   1004  100000004       종로2가.삼일교  126.987507  37.568582
4   1005  100000005  혜화동로터리.여운형활동터  127.001694  37.586230

In [74]:
bus.head()

사용일자      노선ID  노선번호                   노선명  표준버스정류장ID  ARSID      역ID  \
0  20200101  11110371   N62  N62번(양천공영차고지~면목동차고지)  103000034  04133    72645   
1  20200101  41110061  6616         6616(철산동~온수동)  116000120  17210    10202   
2  20200101  41110061  6616         6616(철산동~온수동)  116000080  17166  8001850   
3  20200101  41110061  6616         6616(철산동~온수동)  116000083  17169  9004386   
4  20200101  41110061  6616         6616(철산동~온수동)  116000085  17171  9004384   

              정류장명  승차총승객수  하차총승객수      등록일자  
0             왕십리역       0      13  20200104  
1     현대아파트.개봉중앙시장     153      92  20200104  
2            경인중학교     171      46  20200104  
3  오류1동주민센터.오류문화센터      87      16  20200104  
4             오류동역     129      46  20200104

# bus coord to address

In [72]:
kor_address.head()

시도    시군구  행정구역명 행정동(행정기관명)    법정동   행정구역코드      행정기관코드  행정기관 생성일  \
1  서울특별시  서울특별시  서울특별시      서울특별시  서울특별시       11  1100000000  19880423   
2  서울특별시    종로구    종로구        종로구    종로구    11010  1111000000  19880423   
3  서울특별시    종로구  청운효자동      청운효자동    청운동  1101072  1111051500  20081101   
4  서울특별시    종로구  청운효자동      청운효자동    신교동  1101072  1111051500  20081101   
5  서울특별시    종로구  청운효자동      청운효자동    궁정동  1101072  1111051500  20081101   

        법정동코드 관할지역            행정동 영문명칭  
1  1100000000  NaN               Seoul  
2  1111000000  NaN           Jongno-gu  
3  1111010100  NaN  Cheongunhyoja-dong  
4  1111010200  NaN  Cheongunhyoja-dong  
5  1111010300  NaN  Cheongunhyoja-dong

In [73]:
area_code.head()

code   지역구
0     2    중구
1     3   용산구
2     4   성동구
3     5   광진구
4     6  동대문구

# area code preprocessing

In [61]:
'''지역구 코드와 행정구역 코드 3,4번째 숫자가 같은지 검정'''
seoul = kor_address[kor_address['시도']=='서울특별시']
seoul2 = pd.merge(seoul,area_code,left_on='시군구',right_on='지역구',how='left')

for i in range(1,len(seoul2)):#서울제외
    if int(seoul2['행정구역코드'].iloc[i][2:4]) == int(seoul2['code'].iloc[i]):
        pass
    else:
        print(i)
print('valid finish')

valid finish


# 상권 데이터